# Setup

In [1]:
# imports
import pandas as pd
import numpy as np
import data_describe as dd
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import OneHotEncoder

/home/alex/Software/anaconda3/envs/model-promotion/lib/python3.9/site-packages/xgboost/compat.py:36: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



## Prepare Data

In [2]:
# prepare data
def preprocess(df):
    # clean prediction feature, make binary
    df['income'] = df['income'].str.replace('.', '')
    assert set(df['income']) == set([">50K", "<=50K"])
    
    df['income_gt_50k'] = (df['income']==">50K")
    df = df.drop(['income'], axis='columns')

    # onehot encode remaining categorical variables
    cat_cols = list(set(df.columns) - set(df._get_numeric_data().columns))
    df = pd.get_dummies(df, columns=cat_cols, dtype="int64")
    df['income_gt_50k'] = df['income_gt_50k'].astype(int)

    return df


train_data = pd.read_csv('../data/census_income_census_income_data_adult.data', skipinitialspace=True, comment="|")
test_data = pd.read_csv('../data/census_income_census_income_data_adult.test', skipinitialspace=True, comment="|")

train_data = preprocess(train_data)
test_data = preprocess(test_data)

cols = train_data.columns
train_features = list(set(cols) - set(["income_gt_50k"]))
X_train, y_train = train_data.loc[:, train_data.columns.isin(train_features)], np.array(train_data["income_gt_50k"])
X_test, y_test = test_data.loc[:, test_data.columns.isin(train_features)], np.array(test_data["income_gt_50k"])

for dummy_col in list(set(train_features) - set(X_test.columns)):
    X_test[dummy_col] = 0

/tmp/ipykernel_25133/2196195021.py:4: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.

/tmp/ipykernel_25133/2196195021.py:4: FutureWarning:

The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.



# EDA

In [10]:
X_train.describe()

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,relationship_Husband,relationship_Not-in-family,relationship_Other-relative,relationship_Own-child,...,education_HS-grad,education_Masters,education_Preschool,education_Prof-school,education_Some-college,race_Amer-Indian-Eskimo,race_Asian-Pac-Islander,race_Black,race_Other,race_White
count,32561.000000,3.256100e+04,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,...,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000,32561.000000
mean,38.581647,1.897784e+05,10.080679,1077.648844,87.303830,40.437456,0.405178,0.255060,0.030128,0.155646,...,0.322502,0.052916,0.001566,0.017690,0.223918,0.009551,0.031909,0.095943,0.008323,0.854274
std,13.640433,1.055500e+05,2.572720,7385.292085,402.960219,12.347429,0.490934,0.435901,0.170942,0.362525,...,0.467441,0.223869,0.039546,0.131824,0.416874,0.097264,0.175761,0.294518,0.090851,0.352837
min,17.000000,1.228500e+04,1.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,28.000000,1.178270e+05,9.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,37.000000,1.783560e+05,10.000000,0.000000,0.000000,40.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
75%,48.000000,2.370510e+05,12.000000,0.000000,0.000000,45.000000,1.000000,1.000000,0.000000,0.000000,...,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
max,90.000000,1.484705e+06,16.000000,99999.000000,4356.000000,99.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


# Modeling

In [83]:
model = XGBClassifier()
model.fit(X_train, y_train)
# TODO do cross-validation

/home/alex/Software/anaconda3/envs/model-promotion/lib/python3.9/site-packages/xgboost/sklearn.py:1224: UserWarning:

The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].

/home/alex/Software/anaconda3/envs/model-promotion/lib/python3.9/site-packages/xgboost/data.py:262: FutureWarning:

pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.



[23:24:36] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
              tree_method='exact', validate_parameters=1, verbosity=None)

# Evaluation

In [84]:
model.score(X_test, y_test)

0.8260549106320251

# Serializing